<a href="https://colab.research.google.com/github/Kinrider/TechChallenge/blob/main/ETL_vinhos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [1]:
import pandas as pd

# Define as opções para exibir floats sem notação científica

pd.set_option('display.float_format', '{:.0f}'.format)

# Tratamento de dados de Exportação

In [2]:
# Para ler arquivos do Git, sempre específicar no final : '?raw=true'

dados = 'https://github.com/Kinrider/TechChallenge/blob/cad34f20bb46229c8cc8b78d6a21143aa68d6726/Fontes/ExpVinho.csv?raw=true'

In [3]:
vinhosexp = pd.read_csv(dados, sep=';' , encoding= 'utf8')
vinhosexp
x = vinhosexp.copy()

In [4]:
x.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
array_dollar=['Id' , 'País']
array_kilos=['Id' , 'País']

for column in x.columns:
  if '.1' in column:
   dollar = column.replace('.1' , '')+'_'+'dollar'
   x.rename({column:dollar},inplace=True, axis=1)
   array_dollar.append(dollar)
  else:
    kilos = column +'_'+ 'kilos'
    x.rename({column:kilos},inplace=True, axis=1)
    array_kilos.append(kilos)

In [6]:
x.rename({'Id_kilos' :'Id'},inplace=True, axis=1)
x.rename({'País_kilos':'País'},inplace=True, axis=1)

In [7]:
array_kilos.remove('Id_kilos')
array_kilos.remove('País_kilos')

In [8]:
x[array_kilos].head()

,Id,País,1970_kilos,1971_kilos,1972_kilos,1973_kilos,1974_kilos,1975_kilos,1976_kilos,1977_kilos,...,2014_kilos,2015_kilos,2016_kilos,2017_kilos,2018_kilos,2019_kilos,2020_kilos,2021_kilos,2022_kilos,2023_kilos
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,4,0,0,117
2,3,"Alemanha, República Democrática",0,0,4168,12000,0,0,0,0,...,213348,10680,14012,15467,10794,3660,6261,2698,7630,4806
3,4,Angola,0,0,0,0,0,0,0,0,...,12182,1908,7359,10170,477,345,0,0,4068,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
x[array_dollar].head()

,Id,País,1970_dollar,1971_dollar,1972_dollar,1973_dollar,1974_dollar,1975_dollar,1976_dollar,1977_dollar,...,2014_dollar,2015_dollar,2016_dollar,2017_dollar,2018_dollar,2019_dollar,2020_dollar,2021_dollar,2022_dollar,2023_dollar
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,95,21,0,0,698
2,3,"Alemanha, República Democrática",0,0,2630,8250,0,0,0,0,...,761653,44780,68109,87702,45382,25467,32605,6741,45367,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,23124,17089,35390,61680,709,1065,0,0,4761,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df = x

In [11]:
# Reshape do dataframe

# 1º transpor kilos para linha

df1 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_kilos').columns,
    var_name = 'ano',
    value_name = 'kilos'
    )

# 2º transpor dólar para linha

df2 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_dollar').columns,
    var_name = 'anos',
    value_name = 'dollar'
    )

# 3º sincronizar os índices

df1 = df1.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])
df2 = df2.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])

# 4º Juntar data_frames

df3 = (
    pd.concat([df1, df2], axis=1)
  .sort_index(level=2)
  .reset_index(level=2, drop=True)
  .reset_index()
  )

df3.head()

# referências:
# https://stackoverflow.com/questions/51519101/simultaneously-melt-multiple-columns-in-python-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
# https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87

,Id,País,ano,kilos,anos,dollar
0,1,Afeganistão,1970_kilos,0,1970_dollar,0
1,2,África do Sul,1970_kilos,0,1970_dollar,0
2,3,"Alemanha, República Democrática",1970_kilos,0,1970_dollar,0
3,4,Angola,1970_kilos,0,1970_dollar,0
4,5,Anguilla,1970_kilos,0,1970_dollar,0


In [12]:
# Dropa coluna de ano duplicada

df3.drop(['anos'], axis=1, inplace = True)

In [13]:
# tratando o campo de ano

df3['ano'] = df3['ano'].str.replace('_kilos','') + '-01-01'

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [14]:
# tranformando string em data

df3['ano'] = pd.to_datetime(df3['ano'],format='%Y-%m-%d')

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [15]:
# fitrando dados de 2008 pra frente:

df3.query('ano >= "2008-01-01"' ,inplace = True)

In [16]:
df3.head()

,Id,País,ano,kilos,dollar
5206,1,Afeganistão,2008-01-01,0,0
5207,2,África do Sul,2008-01-01,0,0
5208,3,"Alemanha, República Democrática",2008-01-01,265742,429970
5209,4,Angola,2008-01-01,25721,71083
5210,5,Anguilla,2008-01-01,0,0


In [17]:
# Verificando nomenclatura dos países

df3['País'].unique()

array(['Afeganistão', 'África do Sul', 'Alemanha, República Democrática',
       'Angola', 'Anguilla', 'Antígua e Barbuda', 'Antilhas Holandesas',
       'Arábia Saudita', 'Argentina', 'Aruba', 'Austrália', 'Áustria',
       'Bahamas', 'Bangladesh', 'Barbados', 'Barein', 'Bélgica', 'Belice',
       'Benin', 'Bermudas', 'Bolívia', 'Bósnia-Herzegovina', 'Brasil',
       'Bulgária', 'Cabo Verde', 'Camarões', 'Canadá', 'Catar',
       'Cayman, Ilhas', 'Chile', 'China', 'Chipre', 'Cingapura',
       'Cocos (Keeling), Ilhas', 'Colômbia', 'Comores', 'Congo',
       'Coreia, Republica Sul', 'Costa do Marfim', 'Costa Rica',
       'Croácia', 'Cuba', 'Curaçao', 'Dinamarca', 'Dominica',
       'El Salvador', 'Emirados Arabes Unidos', 'Equador',
       'Eslovaca, Republica', 'Espanha', 'Estados Unidos', 'Estônia',
       'Filipinas', 'Finlândia', 'França', 'Gana', 'Gibraltar', 'Granada',
       'Grécia', 'Guatemala', 'Guiana', 'Guiana Francesa', 'Guine Bissau',
       'Guine Equatorial', 'Haiti', 

In [18]:
# Solicitado ao GPT que gerasse a lista de de_para, considerando os países listados acima

lista_paises = [
    ('Afeganistão', 'Afghanistan', 'AFG'),
    ('África do Sul', 'South Africa', 'ZAF'),
    ('Alemanha, República Democrática', 'Germany', 'DEU'),
    ('Angola', 'Angola', 'AGO'),
    ('Anguilla', 'Anguilla', 'AIA'),
    ('Antígua e Barbuda', 'Antigua and Barbuda', 'ATG'),
    ('Antilhas Holandesas', 'Netherlands Antilles', 'ANT'),
    ('Arábia Saudita', 'Saudi Arabia', 'SAU'),
    ('Argentina', 'Argentina', 'ARG'),
    ('Aruba', 'Aruba', 'ABW'),
    ('Austrália', 'Australia', 'AUS'),
    ('Áustria', 'Austria', 'AUT'),
    ('Bahamas', 'Bahamas', 'BHS'),
    ('Bangladesh', 'Bangladesh', 'BGD'),
    ('Barbados', 'Barbados', 'BRB'),
    ('Barein', 'Bahrain', 'BHR'),
    ('Bélgica', 'Belgium', 'BEL'),
    ('Belice', 'Belize', 'BLZ'),
    ('Benin', 'Benin', 'BEN'),
    ('Bermudas', 'Bermuda', 'BMU'),
    ('Bolívia', 'Bolivia', 'BOL'),
    ('Bósnia-Herzegovina', 'Bosnia and Herzegovina', 'BIH'),
    ('Brasil', 'Brazil', 'BRA'),
    ('Bulgária', 'Bulgaria', 'BGR'),
    ('Cabo Verde', 'Cabo Verde', 'CPV'),
    ('Camarões', 'Cameroon', 'CMR'),
    ('Canadá', 'Canada', 'CAN'),
    ('Catar', 'Qatar', 'QAT'),
    ('Cayman, Ilhas', 'Cayman Islands', 'CYM'),
    ('Chile', 'Chile', 'CHL'),
    ('China', 'China, mainland', 'CHN'),
    ('Chipre', 'Cyprus', 'CYP'),
    ('Cingapura', 'Singapore', 'SGP'),
    ('Cocos (Keeling), Ilhas', 'Cocos (Keeling) Islands', 'CCK'),
    ('Colômbia', 'Colombia', 'COL'),
    ('Comores', 'Comoros', 'COM'),
    ('Congo', 'Congo', 'COG'),
    ('Coreia, Republica Sul', 'Korea (the Republic of)', 'KOR'),
    ('Costa do Marfim', "Côte d'Ivoire", 'CIV'),
    ('Costa Rica', 'Costa Rica', 'CRI'),
    ('Croácia', 'Croatia', 'HRV'),
    ('Cuba', 'Cuba', 'CUB'),
    ('Curaçao', 'Curacao', 'CUW'),
    ('Dinamarca', 'Denmark', 'DNK'),
    ('Dominica', 'Dominica', 'DMA'),
    ('El Salvador', 'El Salvador', 'SLV'),
    ('Emirados Arabes Unidos', 'United Arab Emirates', 'ARE'),
    ('Equador', 'Ecuador', 'ECU'),
    ('Eslovaca, Republica', 'Slovakia', 'SVK'),
    ('Espanha', 'Spain', 'ESP'),
    ('Estados Unidos', 'United States of America', 'USA'),
    ('Estônia', 'Estonia', 'EST'),
    ('Filipinas', 'Philippines', 'PHL'),
    ('Finlândia', 'Finland', 'FIN'),
    ('França', 'France', 'FRA'),
    ('Gana', 'Ghana', 'GHA'),
    ('Gibraltar', 'Gibraltar', 'GIB'),
    ('Granada', 'Grenada', 'GRD'),
    ('Grécia', 'Greece', 'GRC'),
    ('Guatemala', 'Guatemala', 'GTM'),
    ('Guiana', 'Guyana', 'GUY'),
    ('Guiana Francesa', 'French Guiana', 'GUF'),
    ('Guine Bissau', 'Guinea-Bissau', 'GNB'),
    ('Guine Equatorial', 'Equatorial Guinea', 'GNQ'),
    ('Haiti', 'Haiti', 'HTI'),
    ('Honduras', 'Honduras', 'HND'),
    ('Hong Kong', 'China, Hong Kong SAR', 'HKG'),
    ('Hungria', 'Hungary', 'HUN'),
    ('Ilha de Man', 'Isle of Man', 'IMN'),
    ('Ilhas Virgens', 'Virgin Islands', 'VIR'),
    ('India', 'India', 'IND'),
    ('Indonésia', 'Indonesia', 'IDN'),
    ('Irã', 'Iran', 'IRN'),
    ('Iraque', 'Iraq', 'IRQ'),
    ('Irlanda', 'Ireland', 'IRL'),
    ('Itália', 'Italy', 'ITA'),
    ('Jamaica', 'Jamaica', 'JAM'),
    ('Japão', 'Japan', 'JPN'),
    ('Jordânia', 'Jordan', 'JOR'),
    ('Letônia', 'Latvia', 'LVA'),
    ('Líbano', 'Lebanon', 'LBN'),
    ('Libéria', 'Liberia', 'LBR'),
    ('Luxemburgo', 'Luxemburg', 'LUX'),
    ('Macau', 'China, Macao SAR', 'MAC'),
    ('Malásia', 'Malaysia', 'MYS'),
    ('Malavi', 'Malawi', 'MWI'),
    ('Malta', 'Malta', 'MLT'),
    ('Marshall, Ilhas', 'Marshall Islands', 'MHL'),
    ('Martinica', 'Martinique', 'MTQ'),
    ('Mauritânia', 'Mauritania', 'MRT'),
    ('México', 'Mexico', 'MEX'),
    ('Moçambique', 'Mozambique', 'MOZ'),
    ('Montenegro', 'Montenegro', 'MNE'),
    ('Namíbia', 'Namibia', 'NAM'),
    ('Nicarágua', 'Nicaragua', 'NIC'),
    ('Nigéria', 'Nigeria', 'NGA'),
    ('Noruega', 'Norway', 'NOR'),
    ('Nova Caledônia', 'New Caledonia', 'NCL'),
    ('Nova Zelândia', 'New Zealand', 'NZL'),
    ('Omã', 'Oman', 'OMN'),
    ('Países Baixos', 'Netherlands', 'NLD'),
    ('Palau', 'Palau', 'PLW'),
    ('Panamá', 'Panama', 'PAN'),
    ('Paraguai', 'Paraguay', 'PRY'),
    ('Peru', 'Peru', 'PER'),
    ('Pitcairn', 'Pitcairn Islands', 'PCN'),
    ('Polônia', 'Poland', 'POL'),
    ('Porto Rico', 'Puerto Rico', 'PRI'),
    ('Portugal', 'Portugal', 'PRT'),
    ('Quênia', 'Kenya', 'KEN'),
    ('Reino Unido', 'United Kingdom', 'GBR'),
    ('República Dominicana', 'Dominican Republic', 'DOM'),
    ('Rússia', 'Russia', 'RUS'),
    ('São Cristóvão e Névis', 'Saint Kitts and Nevis', 'KNA'),
    ('São Tomé e Príncipe', 'Sao Tome and Principe', 'STP'),
    ('São Vicente e Granadinas', 'Saint Vincent and the Grenadines', 'VCT'),
    ('Senegal', 'Senegal', 'SEN'),
    ('Serra Leoa', 'Sierra Leone', 'SLE'),
    ('Singapura', 'Singapore', 'SGP'),
    ('Suazilândia', 'Eswatini', 'SWZ'),
    ('Suécia', 'Sweden', 'SWE'),
    ('Suíça', 'Switzerland', 'CHE'),
    ('Suriname', 'Suriname', 'SUR'),
    ('Tailândia', 'Thailand', 'THA'),
    ('Taiwan (Formosa)', 'China, Taiwan Province of', 'TWN'),
    ('Tanzânia', 'Tanzania, the United Republic of', 'TZA'),
    ('Tcheca, República', 'Czech Republic', 'CZE'),
    ('Togo', 'Togo', 'TGO'),
    ('Toquelau', 'Tokelau', 'TKL'),
    ('Trinidade Tobago', 'Trinidad and Tobago', 'TTO'),
    ('Tunísia', 'Tunisia', 'TUN'),
    ('Turquia', 'Republic of Türkiye', 'TUR'),
    ('Tuvalu', 'Tuvalu', 'TUV'),
    ('Uruguai', 'Uruguay', 'URY'),
    ('Vanuatu', 'Vanuatu', 'VUT'),
    ('Venezuela', 'Venezuela', 'VEN'),
    ('Vietnã', 'Vietnam', 'VNM')
]

# Tranformando a tupla de países em um dataframe:

de_para_pais = pd.DataFrame(lista_paises, columns = ['pais_pt','pais_in','cod_pais'])

In [19]:
# join com de_para de paises

df_exp = df3.merge(de_para_pais, how = 'left', left_on=['País'],right_on = ['pais_pt'])

In [20]:
# Dropa colunas não utilizadas:

df_exp.drop(['País','pais_in','Id'], axis=1, inplace = True)

In [21]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA


# Tratamento de dados de Consumo

In [22]:
# input de dados de consumo
# Fonte: https://www.oiv.int/what-we-do/statistics

dado_consumo = 'https://github.com/Kinrider/TechChallenge/blob/cad34f20bb46229c8cc8b78d6a21143aa68d6726/Fontes/Consumo.xlsx?raw=true'

In [23]:
# Atribui Dataframe

consumo = pd.read_excel(dado_consumo, sheet_name = 'Export')
consumo.head()

,Continent,Region/Country,Product,Variable,Year,Unit,Quantity
0,Asia,Afghanistan,Dried Grapes,Consumption,1995,tonnes,8050
1,Asia,Afghanistan,Table Grapes,Consumption,1995,tonnes,189800
2,Asia,Afghanistan,Dried Grapes,Consumption,1996,tonnes,8100
3,Asia,Afghanistan,Table Grapes,Consumption,1996,tonnes,176100
4,Asia,Afghanistan,Dried Grapes,Consumption,1997,tonnes,8100


In [24]:
# Renomeia colunas para Português

consumo.rename(columns={'Continent': 'continente', 'Region/Country': 'pais', 'Product': 'produto', 'Variable': 'variavel', 'Year': 'ano','Unit':'unidade','Quantity':'quantidade'}, inplace=True)

In [25]:
# Verificando nomenclatura dos países
# Obs: Nomes estão em inglês, necessário conversão

consumo['pais'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belgium and Luxemburg', 'Belize', 'Benin', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile',
       'China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland',
       'China, Taiwan Province of', 'Colombia', 'Comoros', 'Congo',
       'Congo (the Democratic Republic of the)', 'Cook Islands',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland'

In [26]:
# Verificando lista de produtos
# obs: Apenas vinho é interessante

view2 = consumo['produto'].unique()
view2

array(['Dried Grapes', 'Table Grapes', 'Wine', nan], dtype=object)

In [27]:
# Selecionando apenas vinho

consumo.query("produto == 'Wine'",inplace = True)

In [28]:
# Removendo pais = nan

consumo.dropna(subset = ['pais'], inplace=True)

In [29]:
consumo.head()

,continente,pais,produto,variavel,ano,unidade,quantidade
9268,Asia,Afghanistan,Wine,Consumption,1995,1000 hl,0
9269,Asia,Afghanistan,Wine,Consumption,1996,1000 hl,0
9270,Asia,Afghanistan,Wine,Consumption,1997,1000 hl,0
9271,Asia,Afghanistan,Wine,Consumption,1998,1000 hl,0
9272,Asia,Afghanistan,Wine,Consumption,1999,1000 hl,0


In [30]:
# tratando Data

consumo['ano'] = consumo['ano'].astype(int).astype(str) + '-01-01'

In [31]:
# tranformando string em data

consumo['ano'] = pd.to_datetime(consumo['ano'],format='%Y-%m-%d')

In [32]:
# Filtrando dados desde 2008

consumo.query('ano >= "2008-01-01"', inplace = True)

In [33]:
consumo.head()

,continente,pais,produto,variavel,ano,unidade,quantidade
9281,Asia,Afghanistan,Wine,Consumption,2008-01-01,1000 hl,1
9282,Asia,Afghanistan,Wine,Consumption,2009-01-01,1000 hl,3
9283,Asia,Afghanistan,Wine,Consumption,2010-01-01,1000 hl,2
9284,Asia,Afghanistan,Wine,Consumption,2011-01-01,1000 hl,1
9285,Asia,Afghanistan,Wine,Consumption,2012-01-01,1000 hl,1


In [34]:
# join com de_para de paises

df_cons = consumo.merge(de_para_pais, how = 'left', left_on=['pais'],right_on = ['pais_in'])

In [35]:
# renomeia coluna

df_cons.rename(columns={'quantidade': 'volume_consumido(litros)'}, inplace = True)

In [36]:
# Dropa Colunas não utilizadas

df_cons.drop(['pais','produto','variavel','unidade','pais_pt','pais_in'], axis=1, inplace = True)

In [37]:
df_cons.head()

,continente,ano,volume_consumido(litros),cod_pais
0,Asia,2008-01-01,1,AFG
1,Asia,2009-01-01,3,AFG
2,Asia,2010-01-01,2,AFG
3,Asia,2011-01-01,1,AFG
4,Asia,2012-01-01,1,AFG


# União de Arquivos

In [38]:
# join com de_para de paises

df_vinhos = df_exp.merge(df_cons, how = 'left', left_on=['ano','cod_pais'],right_on = ['ano','cod_pais'])

In [39]:
df_vinhos.head()

,ano,kilos,dollar,pais_pt,cod_pais,continente,volume_consumido(litros)
0,2008-01-01,0,0,Afeganistão,AFG,Asia,1
1,2008-01-01,0,0,África do Sul,ZAF,Africa,3221
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU,Europe,20747
3,2008-01-01,25721,71083,Angola,AGO,Africa,1160
4,2008-01-01,0,0,Anguilla,AIA,NaN,NaN


In [40]:
# converte qauntidade para inteiro e depoiis Hectolitro para litro x mill conforme específicado na coluna

df_vinhos['volume_consumido(litros)'] = df_vinhos['volume_consumido(litros)']*(100*1000)
df_vinhos

,ano,kilos,dollar,pais_pt,cod_pais,continente,volume_consumido(litros)
0,2008-01-01,0,0,Afeganistão,AFG,Asia,100000
1,2008-01-01,0,0,África do Sul,ZAF,Africa,322100000
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU,Europe,2074700000
3,2008-01-01,25721,71083,Angola,AGO,Africa,116000000
4,2008-01-01,0,0,Anguilla,AIA,NaN,NaN
...,...,...,...,...,...,...,...
2217,2023-01-01,0,0,Tuvalu,TUV,NaN,NaN
2218,2023-01-01,326093,454271,Uruguai,URY,America,39400000
2219,2023-01-01,0,0,Vanuatu,VUT,NaN,NaN
2220,2023-01-01,141030,220512,Venezuela,VEN,NaN,NaN


In [41]:
# Preenchendo na com zero

df_vinhos['volume_consumido(litros)'].fillna(0, inplace=True)
df_vinhos

<ipython-input-41-763b0781b81e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_vinhos['volume_consumido(litros)'].fillna(0, inplace=True)


,ano,kilos,dollar,pais_pt,cod_pais,continente,volume_consumido(litros)
0,2008-01-01,0,0,Afeganistão,AFG,Asia,100000
1,2008-01-01,0,0,África do Sul,ZAF,Africa,322100000
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU,Europe,2074700000
3,2008-01-01,25721,71083,Angola,AGO,Africa,116000000
4,2008-01-01,0,0,Anguilla,AIA,NaN,0
...,...,...,...,...,...,...,...
2217,2023-01-01,0,0,Tuvalu,TUV,NaN,0
2218,2023-01-01,326093,454271,Uruguai,URY,America,39400000
2219,2023-01-01,0,0,Vanuatu,VUT,NaN,0
2220,2023-01-01,141030,220512,Venezuela,VEN,NaN,0


In [42]:
# Removendo o index

df_vinhos.reset_index(drop=True, inplace=True)

# Salva Arquivos Gerados

In [44]:
##Salva Arquivo

#saída em csv
df_vinhos.to_csv('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/Dados_tratados.csv',sep = ';' , encoding='utf-8', index = False)

#saída em excel
df_vinhos.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/Dados_tratados.xlsx', index = False)